In [26]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
    using NNlib
	using Einsum
	using PyCall
	
	nn = pyimport("flax")
end

PyCall.PyError: PyCall.PyError("PyImport_ImportModule\n\nThe Python package flax could not be imported by pyimport. Usually this means\nthat you did not install flax in the Python version being used by PyCall.\n\nPyCall is currently configured to use the Julia-specific Python distribution\ninstalled by the Conda.jl package.  To install the flax module, you can\nuse `pyimport_conda(\"flax\", PKG)`, where PKG is the Anaconda\npackage that contains the module flax, or alternatively you can use the\nConda package directly (via `using Conda` followed by `Conda.add` etcetera).\n\nAlternatively, if you want to use a different Python distribution on your\nsystem, such as a system-wide Python (as opposed to the Julia-specific Python),\nyou can re-configure PyCall with that Python.   As explained in the PyCall\ndocumentation, set ENV[\"PYTHON\"] to the path/name of the python executable\nyou want to use, run Pkg.build(\"PyCall\"), and re-launch Julia.\n\n", PyObject(Ptr{PyCall.PyObject_struct} @

In [14]:
function elu_feature_map(x)
  return NN.elu(x) + 1
end

elu_feature_map (generic function with 1 method)

In [18]:
function linear_attention(query, key, value, dropout_rng, dropout_rate, broadcast_dropout,deterministic)

    broadcast_dropout=True
    dropout_rng=None
    dropout_rate=0.
    deterministic=False
    feature_map = elu_feature_map()
    eps=1e-6

    Base.delete!(broadcast_dropout)
    Base.delete!(dropout_rng)
    Base.delete!(dropout_rate)
    Base.delete!(deterministic)

   # Base.@assert key.ndim == query.ndim == value.ndim == 4
   # Base.@assert key.shape[:-1] == value.shape[:-1]
   # Base.@assert (query.shape[0:1] == key.shape[0:1] and query.shape[-1] == key.shape[-1])

    query_mapped = feature_map(query)
    key_mapped = feature_map(key)
    
    @einsum kv[n,h,m,d] = key_mapped[n,s,h,d]*value[n,s,h,m]

    sum_1 = sum(key_mapped, axis=1)+eps

    @einsum z_1[n,l,h] = query_mapped[n,l,h,d]*sum_1[n,h,d]

    z = 1/z_1
    
    @einsum y[n,l,h,m] = query_mapped[n,l,h,d]*kv[n,h,m,d]*z[n,l,h]

    return y
end

linear_attention (generic function with 1 method)

In [ ]:
function LinearAttention
    function apply(
        inputs_q,
        inputs_kv,
        num_heads,
        dtype,
        qkv_features,
        out_features,
        causal_mask,
        padding_mask,
        key_padding_mask,
        segmentation,
        key_segmentation,
        cache,
        broadcast_dropout,
        dropout_rng,
        dropout_rate,
        deterministic,
        precision,
        kernel_init,
        bias_init,
        bias=
    )
    dtype=jnp.float32,
    qkv_features=None,
    out_features=None,
    causal_mask=False,
    padding_mask=None,
    key_padding_mask=None,
    segmentation=None,
    key_segmentation=None,
    cache=None,
    broadcast_dropout=True,
    dropout_rng=None,
    dropout_rate=0.,
    deterministic=False,
    precision=None,
    kernel_init=nn.linear.default_kernel_init, #diese Funktionen gibt es für Julia nichts
    bias_init=nn.initializers.zeros,            # diese auch nicht
    bias=True

    if padding_mask == nothing
        NotImplementedError(
            'Currently, we do not support autoregresive decoding.')
  
       # Base.@assert causal_mask or not cache, (
            'Caching is only support for causal attention.')
    end

    Base.@assert inputs_q.ndim == 3
        
    if inputs_kv == nothing
        inputs_kv = inputs_q
    end
  
      features = out_features || inputs_q.shape[-1]
      qkv_features = qkv_features || inputs_q.shape[-1]

      Base.@assert qkv_features % num_heads == 0, (
        'Memory dimension must be divisible by number of heads.')
    
    head_dim = qkv_features // num_heads

    dense = nn.DenseGeneral.partial(
        axis=-1,
        features=(num_heads, head_dim),
        kernel_init=kernel_init,
        bias_init=bias_init,
        bias=bias,
        precision=precision)

    end
end